In [39]:
import numpy as np
import pandas as pd
import mgrs
import sqlite3

In [40]:
# limit to only the MGRS locations in Southern CA
MGRS_100KM_USED = ['11SMT','11SNT','11SNS','11SMS','11SLT']

# filter out class A fires, which are less than 1/4 acre
CLASSES_USED = ['B','C','D','E','F','G']

def missingByCol(df): #count missing values by column
    for col in df.columns:
        if df[col].isna().sum() != 0:
            print(col, df[col].isna().sum())

def getCalTbl(start='2000-01-01', end='2015-12-31'):
    df = pd.DataFrame({'date': pd.date_range(start, end)})
    return df

Import Data: Need to copy and paste sqllite file from Kaggle into local drive path. FPA_FOD_20170508.sqlite https://www.kaggle.com/rtatman/188-million-us-wildfires

Fires - Target data - Kaggle

In [41]:
conn = sqlite3.connect('C:/Users/balso/Downloads/FPA_FOD_20170508.sqlite')
ca_fires = pd.read_sql_query('select * from fires where State = "CA";', conn) # 

m = mgrs.MGRS() #mgrs API
to_mgrs_10km = []  #precision 1
to_mgrs_100km = []  #precision 0
to_mgrs_lat_10km = []
to_mgrs_lon_10km = []

ca_fires_data = pd.DataFrame()

ca_fires_data['object_id'] = ca_fires['OBJECTID']
ca_fires_data['reporting_unit'] = ca_fires['NWCG_REPORTING_UNIT_NAME']
ca_fires_data['fire_date_num'] = ca_fires['DISCOVERY_DATE']
ca_fires_data['fire_date_cal'] = pd.to_datetime(ca_fires['DISCOVERY_DATE'], unit='D', origin='julian') #convert numeric to calendar date
ca_fires_data['fire_year'] = ca_fires['FIRE_YEAR']
ca_fires_data['fire_month'] = ca_fires_data['fire_date_cal'].dt.month_name()
ca_fires_data['fire_day_of_week'] = ca_fires_data['fire_date_cal'].dt.day_name()
ca_fires_data['fire_class'] = ca_fires['FIRE_SIZE_CLASS']
ca_fires_data['lat'] = ca_fires['LATITUDE']
ca_fires_data['lon'] = ca_fires['LONGITUDE']

for i, j in zip(ca_fires_data['lat'].tolist(), ca_fires_data['lon'].tolist()):
    to_mgrs_10km.append(m.toMGRS(i, j, MGRSPrecision=1)) 
    to_mgrs_100km.append(m.toMGRS(i, j, MGRSPrecision=0)) 

ca_fires_data['mgrs_10km'] = to_mgrs_10km
ca_fires_data['mgrs_100km'] = to_mgrs_100km

for k in ca_fires_data['mgrs_10km'].tolist():
    to_mgrs_lat_10km.append(m.toLatLon(k)[0])
    to_mgrs_lon_10km.append(m.toLatLon(k)[1])

ca_fires_data['mgrs_lat_10km'] = to_mgrs_lat_10km
ca_fires_data['mgrs_lon_10km'] = to_mgrs_lon_10km

# filter out winter months where fires not likely
# ca_fires_data = ca_fires_data[ca_fires_data.fire_month.isin(MONTHS_USED)] #don't apply this filter until later

# filter out class A fires, which are less than 1/4 acre
ca_fires_data = ca_fires_data[ca_fires_data.fire_class.isin(CLASSES_USED)]

# filter out fires before 2000 since weather data is sparse before year 2000
ca_fires_data = ca_fires_data[ca_fires_data.fire_year >= 2000]

# limit to only the MGRS locations in Southern CA
socal_fires_data = ca_fires_data[ca_fires_data.mgrs_100km.isin(MGRS_100KM_USED)]

# keep only necessary columns
socal_lite = pd.DataFrame()
socal_lite['mgrs_100km'] = socal_fires_data['mgrs_100km']
socal_lite['mgrs_10km'] = socal_fires_data['mgrs_10km']
socal_lite['lat'] = socal_fires_data['mgrs_lat_10km']
socal_lite['lon'] = socal_fires_data['mgrs_lon_10km']
socal_lite['date'] = socal_fires_data['fire_date_cal']
socal_lite = socal_lite.sort_values(by = ['mgrs_10km','date'])
socal_lite.reset_index(drop=True, inplace = True)
socal_lite.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9191 entries, 0 to 9190
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   mgrs_100km  9191 non-null   object        
 1   mgrs_10km   9191 non-null   object        
 2   lat         9191 non-null   float64       
 3   lon         9191 non-null   float64       
 4   date        9191 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 359.1+ KB


In [42]:
calendar = getCalTbl()
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5844 entries, 0 to 5843
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    5844 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 45.8 KB


In [43]:
#Import mgrs locations
mgrs_final = pd.read_csv('https://raw.githubusercontent.com/jbalson0808/fires/main/mgrs_final.csv') #set location of mgrs_final
mgrs_final = mgrs_final[['mgrs_10km','lat_10km','lon_10km']]
mgrs_final.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   mgrs_10km  385 non-null    object 
 1   lat_10km   385 non-null    float64
 2   lon_10km   385 non-null    float64
dtypes: float64(2), object(1)
memory usage: 9.1+ KB


In [44]:
mgrs_final['key'] = int(0)
calendar['key'] = int(0)
mgrs_date = mgrs_final.merge(calendar,on='key',how='outer') #false key for cross join
mgrs_date.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2249940 entries, 0 to 2249939
Data columns (total 5 columns):
 #   Column     Dtype         
---  ------     -----         
 0   mgrs_10km  object        
 1   lat_10km   float64       
 2   lon_10km   float64       
 3   key        int64         
 4   date       datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 103.0+ MB


In [45]:
fires = mgrs_date.merge(socal_lite,on=('mgrs_10km','date'),how='left')
is_fire = []
for i in fires['lat']:
    # print(pd.isnull(i))
    if pd.isnull(i) == True:
        is_fire.append(int(0))
    else:
        is_fire.append(int(1))
# np.sum(is_fire)
fires['is_fire'] = is_fire
fires.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2250400 entries, 0 to 2250399
Data columns (total 9 columns):
 #   Column      Dtype         
---  ------      -----         
 0   mgrs_10km   object        
 1   lat_10km    float64       
 2   lon_10km    float64       
 3   key         int64         
 4   date        datetime64[ns]
 5   mgrs_100km  object        
 6   lat         float64       
 7   lon         float64       
 8   is_fire     int64         
dtypes: datetime64[ns](1), float64(4), int64(2), object(2)
memory usage: 171.7+ MB


In [46]:
fires.drop(columns=['key','mgrs_100km','lat','lon'], axis=1,inplace=True)
fires.head()

,mgrs_10km,lat_10km,lon_10km,date,is_fire
0,11SMT03,33.705267,-118.079125,2000-01-01,0
1,11SMT03,33.705267,-118.079125,2000-01-02,0
2,11SMT03,33.705267,-118.079125,2000-01-03,0
3,11SMT03,33.705267,-118.079125,2000-01-04,0
4,11SMT03,33.705267,-118.079125,2000-01-05,0


In [47]:
np.sum(fires['is_fire'])

9190

In [48]:
years = 3
days = 365
roll_days = years * days
fires['mgrs_fires_past3yrs'] = fires.groupby(['mgrs_10km'])['is_fire'].transform(lambda x: x.rolling(roll_days).sum())

In [49]:
fires[fires['is_fire']==1]

,mgrs_10km,lat_10km,lon_10km,date,is_fire,mgrs_fires_past3yrs
5057,11SMT03,33.705267,-118.079125,2013-11-05,1,1.0
9885,11SMT04,33.795444,-118.080255,2011-01-24,1,1.0
10044,11SMT04,33.795444,-118.080255,2011-07-02,1,2.0
10597,11SMT04,33.795444,-118.080255,2013-01-05,1,3.0
11088,11SMT04,33.795444,-118.080255,2014-05-11,1,3.0
...,...,...,...,...,...,...
2202589,11SMS67,33.168058,-117.429006,2013-02-06,1,2.0
2207735,11SMS74,32.897775,-117.320775,2011-03-11,1,1.0
2219728,11SMS81,32.627384,-117.213205,2012-01-10,1,1.0
2219762,11SMS81,32.627384,-117.213205,2012-02-13,1,2.0


In [50]:
print(roll_days)
fires[1090:1100]

1095


,mgrs_10km,lat_10km,lon_10km,date,is_fire,mgrs_fires_past3yrs
1090,11SMT03,33.705267,-118.079125,2002-12-26,0,NaN
1091,11SMT03,33.705267,-118.079125,2002-12-27,0,NaN
1092,11SMT03,33.705267,-118.079125,2002-12-28,0,NaN
1093,11SMT03,33.705267,-118.079125,2002-12-29,0,NaN
1094,11SMT03,33.705267,-118.079125,2002-12-30,0,0.0
1095,11SMT03,33.705267,-118.079125,2002-12-31,0,0.0
1096,11SMT03,33.705267,-118.079125,2003-01-01,0,0.0
1097,11SMT03,33.705267,-118.079125,2003-01-02,0,0.0
1098,11SMT03,33.705267,-118.079125,2003-01-03,0,0.0
1099,11SMT03,33.705267,-118.079125,2003-01-04,0,0.0


In [51]:
# start_date = '2003-01-01'
# fires = fires[fires['date'] >= start_date]
# fires.reset_index(drop=True,inplace=True)
# fires.info()

In [52]:
missingByCol(fires)

mgrs_fires_past3yrs 421190


In [53]:
len(fires.mgrs_10km.unique())

385

In [54]:
len(fires.date.unique())

5844

In [55]:
fires.head()

,mgrs_10km,lat_10km,lon_10km,date,is_fire,mgrs_fires_past3yrs
0,11SMT03,33.705267,-118.079125,2000-01-01,0,NaN
1,11SMT03,33.705267,-118.079125,2000-01-02,0,NaN
2,11SMT03,33.705267,-118.079125,2000-01-03,0,NaN
3,11SMT03,33.705267,-118.079125,2000-01-04,0,NaN
4,11SMT03,33.705267,-118.079125,2000-01-05,0,NaN


In [56]:
fires.tail()

,mgrs_10km,lat_10km,lon_10km,date,is_fire,mgrs_fires_past3yrs
2250395,11SMS57,33.167642,-117.536254,2015-12-27,0,0.0
2250396,11SMS57,33.167642,-117.536254,2015-12-28,0,0.0
2250397,11SMS57,33.167642,-117.536254,2015-12-29,0,0.0
2250398,11SMS57,33.167642,-117.536254,2015-12-30,0,0.0
2250399,11SMS57,33.167642,-117.536254,2015-12-31,0,0.0


In [57]:
fires.to_csv('fires_clean.csv',index=False)